In [4]:
#sensor = 'LANDSAT8';
sensor = 'SENTINEL';
dataInicial = '2012-01-01'; #ano,mes,dia
dataFinal = '2020-01-01';
max_nuvens = 5

'''
saoFranciscoFalso,saoFranciscoVerdadeiro,ocoi,

bracos = ['aguaVerde','apipu','bangu','belaVista','doisIrmaos','guabiroba','guacu','itacora220',
              'itavo220','jacutinga','maya','moinho','moledo220','naranjito','ocoi','onca','passoCue',
              'petronius','porvenir','rioBranco','salamanca','sangaAlegre','saoCristovao','saoFranciscoFalso',
              'saoFranciscoVerdadeiro','saoJorge','saoLuiz','saoVicenteChico','saoVicenteSaoJoao','solMaio220',
              'taquapetinga','tigre220','tresIrmas','vinitu','xororo','costanera','dosHermanas',
              'itabo','limoy','pikyry','piraPyta','tatiYupi','brillante','carapa','gasory','itambey','pozuelo','santaTeresa','corpoCentral'] # 49
'''
bracos = ['saoFranciscoFalso','saoFranciscoVerdadeiro'] 

In [5]:
import ee
import psycopg2
import time

print("Iniciando conexão com EE...");
ee.Initialize();
print("Conectado!");

Iniciando conexão com EE...
Conectado!


In [6]:
satelite = -1
if sensor == 'SENTINEL':
    satelite = 1
    bandas = ['B8','B4','B11']
    scale = 10
    clouds = 'CLOUDY_PIXEL_PERCENTAGE'
    dataset = 'COPERNICUS/S2'
    
    escolha = 0
    print 'Sentinel escolhido!'
else:
    satelite = 0
    bandas = ['B5','B4','B6']
    scale = 30
    clouds = 'CLOUD_COVER'
    dataset = 'LANDSAT/LC08/C01/T1_SR'
    
    escolha = 1
    print 'Landsat escolhido!'

Sentinel escolhido!


In [7]:
dataset = ee.ImageCollection(dataset) \
            .filter(ee.Filter.date(dataInicial, dataFinal)) \
            .filterBounds(ee.Algorithms.GeometryConstructors.MultiPoint([[-54.32822528851034, -24.660395975306944],[-54.49576679241659, -25.30643825264147]])) \
            .filter(ee.Filter.lessThanOrEquals(clouds,80));
            
projection = ee.Image(dataset.first()).select('B4').projection();

In [8]:
def mapCloudscore(image):
    if(satelite):
        pq = image.select('QA60');
        return ee.Image(0).where(pq.gt(0),1).rename(ee.List(['cloudMask']));
    else:
        pq = image.select('pixel_qa');
        return ee.Image(0).where(pq.neq(324).And(pq.neq(322)),1).rename(ee.List(['cloudMask']));

In [9]:
def mapNDVI(image):
    image = ee.Image(image);
    ndvi = image.normalizedDifference([bandas[0],bandas[1]]).rename(['ndvi']);
    return ndvi;

In [10]:
def mapMacrofitas(image,projection):
    image = ee.Image(image);
    ndvi = image.normalizedDifference([bandas[0],bandas[1]]);
    soma = ndvi.reproject(projection).reduceResolution(ee.Reducer.sum());
    porcentagem = soma.divide(ndvi).multiply(100).round();
    clippedNdvi = ndvi.updateMask(ndvi.where(porcentagem.lte(99),0));
    output = ee.Image(0).where(clippedNdvi.gte(0.2).And(image.select(bandas[2]).gt(0.06)),1);
    macrofitas = output.updateMask(output).rename(['macrofitas']).reproject(projection);

    return macrofitas;

In [11]:
def getDataStr(image):
    image = ee.Image(image);
    eedate = ee.Date(image.get('system:time_start'))
    dia = ee.Number(eedate.get('day')).format()
    mes = ee.Number(eedate.get('month')).format()
    ano = ee.Number(eedate.get('year')).format()
    data = ano.cat('-').cat(mes).cat('-').cat(dia)
    return data

In [12]:
def mapper(image):

    image = ee.Image(image);
    cloudscore = mapCloudscore(image);   
    macrofitas = ee.Image(mapMacrofitas(image,projection));
    ndvi = ee.Image(mapNDVI(image));
    data = getDataStr(image);

    return image.addBands(macrofitas).addBands(cloudscore).addBands(ndvi).set('data',data);

In [ ]:
collection = dataset.map(mapper);
tamanho = ee.ImageCollection(collection).toList(1000).length().getInfo()
collection = ee.ImageCollection(collection).toList(tamanho);
print str(tamanho)+' imagens\n'

283 imagens



In [ ]:
for i in range(tamanho):
    image = ee.Image(collection.get(i))
    id_imagem = image.id().getInfo()
    data = image.get('data').getInfo()
    
    start_time = time.time()
    bracosInseridos = 0
    print("*================================================================*")
    print "("+str(i+1)+")", id_imagem
    print("*================================================================*")
    
    for i in range(0, len(bracos)):
        braco = bracos[i]
        regiao = ee.Feature(ee.FeatureCollection("users/odraitaipu/bracos/"+braco).union().first()).set('nome',braco).geometry()
        
        intersecao = image.geometry().intersection(regiao,ee.ErrorMargin(1));
        porcentagem_intersecao = ee.Number((intersecao.area().multiply(100)).divide(regiao.area())).getInfo()
        
        print braco
        if porcentagem_intersecao > 10:
            
            meanClouds = ee.Number(image.select('cloudMask').reduceRegion(ee.Reducer.mean(), regiao, scale, None, None, True).get('cloudMask')).multiply(100).getInfo()
            
            if meanClouds <= max_nuvens:
                
                if i < 35:
                    local_segmento = 'me'
                elif i >= 35 and i != 48:
                    local_segmento = 'md'
                elif i == 48:
                    local_segmento = 'cc'

                shapeMacrofita = image.select('macrofitas').reduceToVectors(ee.Reducer.countEvery(),regiao,None,'polygon',True,'macrofitas',None,None,True).union(1).geometry()
                meanNDVI = image.select('ndvi').reduceRegion(ee.Reducer.mean(), shapeMacrofita, scale, None, None, True).get('ndvi')

                areaRegiao = regiao.area();
                areaMacrofitas = shapeMacrofita.area(ee.ErrorMargin(1));
                macrofitas_porcentagem = (areaMacrofitas.multiply(100)).divide(areaRegiao);

                shapeMacrofita = shapeMacrofita.getInfo()
                areaMacrofitas = areaMacrofitas.getInfo()
                macrofitas_porcentagem = macrofitas_porcentagem.getInfo()
                meanNDVI = meanNDVI.getInfo()

                import json
                geoJson = shapeMacrofita
                geoJson["crs"] = {"type": "name","properties": {"name": "EPSG:4326"}}
                geoJsonStr = json.dumps(shapeMacrofita)

                host = '179.106.229.170'
                port = '5432'
                dbname = 'postgis'
                usuario = 'postgres'
                senha = 'postgres'

                comando = "INSERT INTO macrofitas(id,the_geom,data_passagem,sensor,media_ndvi,nome_segmento,local_segmento,area_macrofitas_m2,porcentagem_macrofitas,porcentagem_nuvens,porcentagem_intersecao,id_imagem)" + \
                          "VALUES(DEFAULT,ST_GeomFromGeoJSON('"+geoJsonStr+"'),'"+str(data)+"','"+sensor+"',"+str(meanNDVI)+",'"+braco+"', '"+local_segmento+"',"+str(areaMacrofitas)+","+str(macrofitas_porcentagem)+ \
                          ","+str(meanClouds)+","+str(porcentagem_intersecao)+",'"+id_imagem+"');"

                con = None
                try:
                    con = psycopg2.connect("host='"+host+"' port='"+port+"' dbname='"+dbname+"' user='"+usuario+"' password='"+senha+"'")   
                    cur = con.cursor()
                    cur.execute(comando)
                    con.commit()
                    print 'Inserido!\n'

                    bracosInseridos += 1
                except psycopg2.DatabaseError, e:
                    if con:
                        con.rollback()
                    print 'Error %s' % e
                    bracosInseridos -= 1
                finally:   
                    if con: 
                        con.close()
            else:
                print "Pulando região, "+ str(meanClouds) +"% de cobertura de nuvens\n"
        else:
            print "Pulando região, não está contida na cena\n"
                        
    elapsed_time = time.time() - start_time
    print 'Varredura de imagem finalizada. Inseridos '+str(bracosInseridos)+' bracos em '+ str(elapsed_time) + ' segundos'
    print "("+str((100*i)/tamanho)+"%)"
    print "------------------------------------------------------------------\n\n"
    
print "\n\n/------------------------------------------------------------------------------------------------------------/"
print "Procedimento completo!"
print "/------------------------------------------------------------------------------------------------------------/"

*================================================================*
(1) 20150909T134916_20150909T134918_T21JYM
*================================================================*
saoFranciscoFalso
Pulando região, não está contida na cena

saoFranciscoVerdadeiro
Pulando região, não está contida na cena

Varredura de imagem finalizada. Inseridos 0 bracos em 0.0774860183398 minutos
------------------------------------------------------------------


*================================================================*
(2) 20150909T134916_20150909T134918_T21JYN
*================================================================*
saoFranciscoFalso
Pulando região, 13.7372055046% de cobertura de nuvens

saoFranciscoVerdadeiro
Inserido!

Varredura de imagem finalizada. Inseridos 1 bracos em 0.570062434673 minutos
------------------------------------------------------------------


*================================================================*
(3) 20150909T134916_20161014T075239_T21JYM
*==========